In [1]:
%pip install gdown
import gdown

url = "https://drive.google.com/drive/folders/1_z2a2W9ZmOhjz9FeNhn5GftGC-G-31N9?usp=sharing"
gdown.download_folder(url, quiet=True)

['/content/biometric_systems_project/dataset.zip']

In [ ]:
import zipfile
import os
import cv2
import shutil
import numpy as np
from google.colab.patches import cv2_imshow
from tqdm import tqdm

def main():
    def update_dictionary(key, dictionary, file_name):
        if key not in dictionary:
            dictionary[key] = {file_name}
        else:
            dictionary[key].add(file_name)

    folder_path = "/content/biometric_systems_project"
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".zip"):
            file_path = os.path.join(folder_path, file_name)
            extract_folder = os.path.join(folder_path, file_name.replace(".zip", ""))
            os.makedirs(extract_folder, exist_ok=True)
            with zipfile.ZipFile(file_path, mode="r") as zip_ref:
                zip_ref.extractall(extract_folder)
                for root, _, files in os.walk(extract_folder):
                    for image_file in files:
                        if image_file.lower().endswith('.jpg'):
                            source_file = os.path.join(root, image_file)
                            destination_file = os.path.join(folder_path, image_file)
                            shutil.move(source_file, destination_file)
            shutil.rmtree(extract_folder)
            os.remove(extract_folder + ".zip")

    ages, genders, ethnicities = dict(), dict(), dict()
    for file_name in os.listdir(folder_path):
        if(file_name[0] != "h"):
            if (file_name[0] == "."):
                file_name = file_name[2:]
            else:
                age, gender, ethnicity, _ = file_name.split("_")
                update_dictionary(age, ages, file_name)
                update_dictionary(gender, genders, file_name)
                update_dictionary(ethnicity, ethnicities, file_name)

    face_cascade_path = "/content/biometric_systems_project/haarcascade_frontalface_default.xml"
    face_cascade = cv2.CascadeClassifier(face_cascade_path)

    if face_cascade.empty():
        print("Errore: il file haarcascade_frontalface_default.xml non è stato trovato o caricato correttamente.")
        return

    total_images = len([name for name in os.listdir(folder_path) if name.lower().endswith('.jpg')])

    for file_name in tqdm(os.listdir(folder_path), desc="Elaborating images", total=total_images):
        image_path = os.path.join(folder_path, file_name)
        image = cv2.imread(image_path)
        if image is None:
            continue
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            face_roi = gray_image[y:y+h, x:x+w]

            # Calcola le sezioni del volto usando proporzioni modificate
            upper_part = y + int(h * 0.4)     # Limite inferiore della parte occhi (40%)
            middle_part = y + int(h * 0.7)    # Limite inferiore della parte naso (70%)

            # Disegna le linee per suddividere il volto in 3 parti
            cv2.line(image, (x, upper_part), (x + w, upper_part), (0, 255, 0), 2)    # Linea occhi-naso
            cv2.line(image, (x, middle_part), (x + w, middle_part), (0, 0, 255), 2)  # Linea naso-bocca

            # Etichetta le sezioni
            cv2.putText(image, "Eyes", (x, y + int(h * 0.15)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.putText(image, "Nose", (x, upper_part + int(h * 0.05)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            cv2.putText(image, "Mouth", (x, middle_part + int(h * 0.05)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

            cv2_imshow(image)
            cv2.waitKey(500)

if __name__ == "__main__":
    main()
